In [ ]:
pip install lightgbm

In [19]:
import pandas as pd
import numpy as np

import lightgbm as lgb


from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    roc_auc_score,

)

In [4]:
# Load your generated dataset
df = pd.read_csv("pharmacy_dataset_improved_v2.csv")

# Convert date to datetime
df["date"] = pd.to_datetime(df["date"])


In [7]:
df.head(10)

,record_id,date,pharmacy_id,medicine_id,medicine_category,pharmacy_location_code,supplier_count,current_stock_level,avg_weekly_sales,reorder_quantity,...,price_change_rate,storage_capacity,previous_shortage_count,dos_per_patient,category_shortage_rate,stock_to_sales_ratio,demand_volatility,seasonal_demand_factor,num_patients,target_stockout
0,1,2024-09-06,89,11,Oncology,10,3,51,14,47,...,0.02,265,0,1.59,0.0,3.64,0.64,1.0,16,1
1,2,2024-09-23,98,12,Cardiology,4,1,119,39,100,...,0.20,265,0,0.93,0.0,3.05,0.23,1.0,23,1
2,3,2024-10-07,63,19,Diabetes,12,3,80,44,100,...,-0.03,468,0,0.49,0.0,1.82,0.37,1.0,26,1
3,4,2024-01-21,58,25,Oncology,12,1,78,38,100,...,0.05,304,0,0.60,0.0,2.05,0.63,1.0,24,1
4,5,2024-04-29,33,3,Cardiology,11,1,83,18,42,...,0.11,242,0,1.29,0.0,4.61,0.87,1.0,25,1
5,6,2024-08-14,82,6,Antibiotic,4,3,128,27,76,...,0.04,428,0,1.58,0.0,4.74,0.28,1.0,21,1
6,7,2024-07-30,89,25,Painkiller,4,4,184,52,100,...,0.10,359,0,1.77,0.0,3.54,0.20,1.2,14,1
7,8,2024-05-31,53,26,Antibiotic,10,4,53,46,100,...,0.12,162,0,0.30,0.0,1.15,0.29,1.0,27,1
8,9,2024-06-04,90,34,Diabetes,3,3,128,21,64,...,-0.07,252,0,2.51,0.0,6.10,0.46,1.0,17,1
9,10,2024-06-26,54,48,Oncology,8,2,133,21,65,...,0.13,366,0,3.69,0.0,6.33,0.63,1.0,12,1


In [46]:
df.corr()["target_stockout"].sort_values(ascending=False)

,target_stockout
target_stockout,1.000000
category_shortage_rate,0.273491
record_id,0.177038
supplier_delay_frequency,0.105220
avg_weekly_sales,0.095511
num_patients,0.074455
reorder_quantity,0.072495
demand_volatility,0.048525
previous_shortage_count,0.045494
lead_time_days,0.037766


In [8]:
df.isnull().sum()

,0
record_id,0
date,0
pharmacy_id,0
medicine_id,0
medicine_category,0
pharmacy_location_code,0
supplier_count,0
current_stock_level,0
avg_weekly_sales,0
reorder_quantity,0


In [10]:
# Sort chronologically to avoid data leakage
df = df.sort_values("date").reset_index(drop=True)

In [11]:
df.head()

,record_id,date,pharmacy_id,medicine_id,medicine_category,pharmacy_location_code,supplier_count,current_stock_level,avg_weekly_sales,reorder_quantity,...,price_change_rate,storage_capacity,previous_shortage_count,dos_per_patient,category_shortage_rate,stock_to_sales_ratio,demand_volatility,seasonal_demand_factor,num_patients,target_stockout
0,1526,2024-01-01,32,12,Diabetes,16,2,185,31,100,...,0.13,122,0,1.61,1.0,5.97,0.30,1.0,26,1
1,224,2024-01-01,32,34,Antibiotic,11,1,64,28,96,...,0.09,471,0,1.45,0.3,2.29,0.13,1.4,11,1
2,1115,2024-01-01,3,30,Oncology,12,3,78,37,93,...,0.01,343,0,0.92,1.0,2.11,0.44,1.0,16,1
3,67,2024-01-01,33,30,Antibiotic,5,3,74,33,100,...,-0.01,473,0,1.43,0.2,2.24,0.70,1.4,11,1
4,1562,2024-01-01,56,19,Painkiller,16,2,0,83,100,...,0.03,190,0,0.00,1.0,0.00,0.64,1.0,21,1


In [14]:
TARGET = "target_stockout"

FEATURES = [
    # Identifiers (keep pharmacy & medicine IDs)
    "pharmacy_id",
    "medicine_id",

    # Core inventory & demand features
    "current_stock_level",
    "avg_weekly_sales",
    "reorder_quantity",
    "lead_time_days",
    "supplier_count",
    "supplier_delay_frequency",

    # Economic & capacity features
    "price_change_rate",
    "storage_capacity",

    # Research-based engineered features (Pall et al.)
    "previous_shortage_count",
    "dos_per_patient",
    "category_shortage_rate",
    "stock_to_sales_ratio",
    "demand_volatility",
    "seasonal_demand_factor",
    "num_patients",

    # Contextual features
    "medicine_category",
    "pharmacy_location_code"
]

X = df[FEATURES]
y = df[TARGET]


In [17]:
# Time-based split (example)
train_end = "2024-09-30"
val_end = "2024-10-31"

X_train = X[df["date"] <= train_end]
y_train = y[df["date"] <= train_end]

X_val = X[(df["date"] > train_end) & (df["date"] <= val_end)]
y_val = y[(df["date"] > train_end) & (df["date"] <= val_end)]

X_test = X[df["date"] > val_end]
y_test = y[df["date"] > val_end]

print("Train size:", X_train.shape)
print("Validation size:", X_val.shape)
print("Test size:", X_test.shape)


Train size: (2238, 19)
Validation size: (259, 19)
Test size: (503, 19)


In [31]:
model = lgb.LGBMClassifier(
    objective="binary",
    n_estimators=100,
    learning_rate=0.05,
    max_depth=8,
    num_leaves=63,
    random_state=42,
    class_weight="balanced"  # IMPORTANT for shortage imbalance
)


In [32]:
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric="auc", # used auc instead of f1 because auc works on probabilities
    callbacks=[lgb.early_stopping(stopping_rounds=20)]
)



[LightGBM] [Info] Number of positive: 2151, number of negative: 87
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000572 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1668
[LightGBM] [Info] Number of data points in the train set: 2238, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(class_weight='balanced', learning_rate=0.05, max_depth=8,
               num_leaves=63, objective='binary', random_state=42)

In [35]:
#model.best_iteration_

4

In [43]:
# Predictions
y_pred = model.predict(
    X_test,
    num_iteration=model.best_iteration_ # cause we used early stopping earlier
                       )
y_proba = model.predict_proba(
    X_test,
    num_iteration=model.best_iteration_
                             )[:, 1]

#print("Predictions:", y_pred)
#print("Probabilities:", y_proba)

In [44]:
# Metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print("📊 TEST SET PERFORMANCE")
print(f"Accuracy : {accuracy:.3f}")
print(f"F1-score : {f1:.3f}")
print(f"Recall   : {recall:.3f}")
print(f"ROC-AUC  : {roc_auc:.3f}")



📊 TEST SET PERFORMANCE
Accuracy : 0.932
F1-score : 0.964
Recall   : 0.944
ROC-AUC  : 0.893
